# Environment setting up

In [1]:
# # Enviroment setting up
# !pip install pandas dash
!pip install jupyter-dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


# Get data

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


--2025-01-01 14:28:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-01-01 14:28:27 (557 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



# Dashboard

In [4]:
# Import required libraries
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Read the SpaceX launch data into a pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = Dash(__name__)

# Define the application layout
app.layout = html.Div(children=[
    # Title
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown for selecting launch sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                 ],
                 value='ALL',  # Default value
                 placeholder="Select a launch site",
                 searchable=True),
    html.Br(),

    # Pie chart for success counts
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[min_payload, max_payload]),
    html.Br(),

    # Scatter plot for payload and success correlation
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


# Callback for updating the pie chart based on selected site
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def render_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Aggregate success counts for each site
        success_counts = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts()
        if success_counts.empty:
            return go.Figure()  # Return an empty figure if no data
        fig = px.pie(values=success_counts.values, names=success_counts.index,
                     title='Launch Success Distribution Across All Sites')
    else:
        # Filter for the selected site and calculate success/failure counts
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_counts = filtered_df['class'].value_counts()
        if success_counts.empty:
            return go.Figure()  # Return an empty figure if no data
        fig = px.pie(values=success_counts.values, names=success_counts.index,
                     title=f'Launch Success for {entered_site}')
    return fig


# Callback for updating the scatter plot based on site and payload range
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def scatter_plot(entered_site, slider_payload):
    # Filter dataframe by payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= slider_payload[0]) &
                            (spacex_df['Payload Mass (kg)'] <= slider_payload[1])]
    if entered_site != 'ALL':
        # Further filter by launch site
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    # Handle case with no data
    if filtered_df.empty:
        return go.Figure()  # Return an empty figure if no data matches

    # Create scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category',
                     title=f'Payload vs. Outcome for {"All Sites" if entered_site == "ALL" else entered_site}',
                     labels={'class': 'Launch Outcome'})
    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>